# Lakeflow Expectations: Data Quality ✅

Welcome! This demo teaches you how to enforce data quality in Lakeflow pipelines using **expectations**.

---

## 🎯 What You'll Learn

By the end of this demo, you will be able to:

1. ✅ Understand what expectations are and why they matter
2. ✅ Use `expect()` to track data quality violations
3. ✅ Use `expect_or_drop()` to filter invalid records
4. ✅ Use `expect_or_fail()` to halt pipelines on critical issues
5. ✅ Combine multiple expectations on a single table
6. ✅ Monitor data quality metrics in the pipeline UI
7. ✅ Apply expectations at different layers (bronze, silver, gold)
8. ✅ Follow data quality best practices

---

## 🔗 Prerequisites

This demo builds on **Lakeflow Pipeline Fundamentals**.

You should already understand:
* ✅ Streaming tables vs materialized views
* ✅ Bronze → Silver → Gold architecture
* ✅ `@dlt.table` and `@dlt.view` decorators
* ✅ Creating and running pipelines

**If you haven't completed the fundamentals demo, do that first!**

---

## 📊 Demo Scenario: Data Quality for Wanderbricks

We'll add data quality checks to the Wanderbricks pipeline:

**Quality checks:**
* Validate booking amounts are positive
* Ensure dates are valid
* Check ratings are in valid range (1-5)
* Verify required fields are not null
* Monitor data quality metrics

**Expectation actions:**
* **Track** violations (warnings)
* **Drop** invalid records
* **Fail** pipeline on critical issues

---

**Let's add data quality to your pipeline!** 🚀

## Part 1: Understanding Expectations 📊

**Expectations** are data quality checks built into your Lakeflow pipeline.

---

### **What are expectations?**

**Definition:**
* Declarative data quality rules
* SQL-like conditions that data must meet
* Applied to streaming tables and views
* Monitored and tracked automatically

**Example:**
```python
@dlt.table(
    expect={"valid_amount": "total_amount > 0"}
)
def silver_bookings():
    return dlt.read_stream("bronze_bookings")
```

**What this does:**
* Checks that `total_amount > 0` for every row
* Tracks violations
* Takes action based on expectation type

---

### **Why use expectations?**

**Without expectations:**
* ❌ Bad data flows through pipeline
* ❌ Issues discovered late (in dashboards)
* ❌ Manual data quality checks
* ❌ No visibility into data quality

**With expectations:**
* ✅ Catch issues early
* ✅ Automatic data quality monitoring
* ✅ Prevent bad data from propagating
* ✅ Built-in metrics and reporting
* ✅ Self-documenting data quality rules

---

### **Where to use expectations:**

* 🥉 **Bronze:** Minimal (track issues, don't drop)
* 🥈 **Silver:** Extensive (drop invalid data)
* 🥇 **Gold:** Critical checks (fail on issues)

---

### 🎯 Three Types of Expectations

Lakeflow has **three expectation functions** with different behaviors:

---

### **1. `expect()` - Track violations (Warning)**

**Behavior:**
* ⚠️ Records violation in metrics
* ✅ Keeps the invalid record
* ✅ Pipeline continues
* 📊 Tracks percentage of violations

**When to use:**
* Bronze layer (track issues, keep all data)
* Non-critical validations
* Monitoring data quality trends

**Syntax:**
```python
@dlt.table(
    expect={"valid_amount": "total_amount > 0"}
)
```

**Result:** Invalid records kept, violations tracked

---

### **2. `expect_or_drop()` - Drop invalid records**

**Behavior:**
* ❌ Drops records that violate the rule
* 📊 Records violation in metrics
* ✅ Pipeline continues
* ✅ Only valid records in output

**When to use:**
* Silver layer (clean data)
* Filter out bad data
* Ensure data quality

**Syntax:**
```python
@dlt.table(
    expect_or_drop={"valid_amount": "total_amount > 0"}
)
```

**Result:** Invalid records dropped, violations tracked

---

### **3. `expect_or_fail()` - Fail pipeline**

**Behavior:**
* 🛑 Stops pipeline execution
* 🔴 Marks table as failed
* ❌ No data written
* 🚨 Requires immediate attention

**When to use:**
* Gold layer (critical metrics)
* Critical business rules
* Regulatory requirements
* When bad data is unacceptable

**Syntax:**
```python
@dlt.table(
    expect_or_fail={"valid_amount": "total_amount > 0"}
)
```

**Result:** Pipeline fails if ANY record violates

---

### **Comparison:**

| Function | Invalid Records | Pipeline | Use Case |
|----------|----------------|----------|----------|
| `expect()` | Kept | Continues | Track issues |
| `expect_or_drop()` | Dropped | Continues | Filter bad data |
| `expect_or_fail()` | None written | Fails | Critical rules |

---

**💡 Tip:** Start with `expect()` to understand your data, then use `expect_or_drop()` or `expect_or_fail()`

## Part 2: `expect()` - Track Data Quality Issues ⚠️

The `expect()` function tracks violations but keeps all records.

**Use case:** Monitor data quality without dropping data (especially in bronze layer)

---

In [0]:
import dlt
from pyspark.sql.functions import *

# ============================================
# BRONZE LAYER with expect()
# ============================================

@dlt.table(
    name="bronze_bookings_monitored",
    comment="Raw bookings with data quality monitoring",
    expect={
        "valid_amount": "total_amount > 0",
        "valid_guests": "guests_count > 0",
        "valid_dates": "check_in < check_out"
    }
)
def bronze_bookings_monitored():
    """
    Bronze table with expectations that TRACK violations.
    Invalid records are KEPT but violations are recorded.
    """
    return (
        spark.readStream
            .table("samples.wanderbricks.bookings")
    )

### 💡 `expect()` - How It Works

**What we did:**

1. **Added `expect` parameter** to `@dlt.table`
2. **Defined 3 expectations:**
   * `valid_amount` - Amount must be positive
   * `valid_guests` - Guest count must be positive
   * `valid_dates` - Check-in before check-out

3. **Each expectation has:**
   * **Name** - Descriptive identifier ("valid_amount")
   * **Condition** - SQL expression ("total_amount > 0")

**What happens:**
* ✅ All records are kept (even invalid ones)
* 📊 Violations are tracked in metrics
* ⚠️ Pipeline shows warning if violations exist
* 📈 You can see violation percentage

**When to use `expect()`:**
* 🥉 **Bronze layer** - Track issues, keep all data for audit
* 📈 **Monitoring** - Understand data quality trends
* 🔍 **Discovery** - Learn about your data
* 📊 **Reporting** - Track quality metrics over time

**Expectation syntax:**
```python
expect={
    "expectation_name": "SQL_condition",
    "another_check": "column IS NOT NULL",
    "range_check": "value BETWEEN 1 AND 100"
}
```

**Valid SQL conditions:**
* Comparison: `>`, `<`, `>=`, `<=`, `=`, `!=`
* NULL checks: `IS NOT NULL`, `IS NULL`
* Range: `BETWEEN x AND y`
* Pattern: `LIKE 'pattern'`
* Complex: `(condition1) AND (condition2)`

---

**💡 Tip:** Use `expect()` in bronze to understand data quality before deciding on stricter rules

### 🎯 Challenge 1: Add expect() to Bronze Reviews

**Your task:**

Add expectations to track data quality in bronze reviews.

**Requirements:**

1. Create a bronze table: `bronze_reviews_monitored`
2. Ingest from `samples.wanderbricks.reviews`
3. Add these expectations using `expect`:
   * `valid_rating` - Rating between 1.0 and 5.0
   * `has_rating` - Rating is not null
   * `not_deleted` - is_deleted = false

**Expected code:**
```python
@dlt.table(
    name="bronze_reviews_monitored",
    comment="Raw reviews with quality monitoring",
    expect={
        "valid_rating": "rating BETWEEN 1.0 AND 5.0",
        "has_rating": "rating IS NOT NULL",
        "not_deleted": "is_deleted = false"
    }
)
def bronze_reviews_monitored():
    return spark.readStream.table("samples.wanderbricks.reviews")
```

**After adding:**
1. Add the code to your notebook
2. Save the notebook
3. Run the pipeline
4. Check the graph - node should show expectation metrics
5. Click the node to see violation counts

✅ **Success criteria:** Bronze table with 3 expectations tracking violations

---

## Part 3: `expect_or_drop()` - Filter Invalid Data ❌

The `expect_or_drop()` function removes records that violate expectations.

**Use case:** Clean data in silver layer by dropping invalid records

---

In [0]:
# ============================================
# SILVER LAYER with expect_or_drop()
# ============================================

@dlt.table(
    name="silver_bookings_quality",
    comment="Cleaned bookings with data quality enforcement - Silver layer",
    expect_or_drop={
        "valid_amount": "total_amount > 0",
        "valid_guests": "guests_count > 0 AND guests_count <= 20",
        "valid_dates": "check_in < check_out",
        "future_checkin": "check_in >= '2020-01-01'",
        "confirmed_status": "status = 'confirmed'"
    }
)
def silver_bookings_quality():
    """
    Silver table with expect_or_drop.
    Records violating ANY expectation are DROPPED.
    Only clean data flows to downstream tables.
    """
    return (
        dlt.read_stream("bronze_bookings")
        .withColumn("nights", datediff(col("check_out"), col("check_in")))
        .withColumn("booking_date", to_date(col("created_at")))
        .withColumn("revenue_per_night", col("total_amount") / col("nights"))
        .select(
            "booking_id",
            "user_id",
            "property_id",
            "check_in",
            "check_out",
            "nights",
            "guests_count",
            "total_amount",
            "revenue_per_night",
            "status",
            "booking_date",
            "created_at"
        )
    )

### 💡 `expect_or_drop()` - How It Works

**What we did:**

1. **Added `expect_or_drop` parameter** to `@dlt.table`
2. **Defined 5 expectations:**
   * `valid_amount` - Positive amounts only
   * `valid_guests` - Reasonable guest count (1-20)
   * `valid_dates` - Check-in before check-out
   * `future_checkin` - No historical bookings before 2020
   * `confirmed_status` - Only confirmed bookings

**What happens:**
* ❌ Records violating ANY expectation are **dropped**
* 📊 Violations are tracked in metrics
* ✅ Only valid records in output table
* ✅ Pipeline continues (doesn't fail)

**Data flow:**
```
Bronze: 10,000 records (all data)
    ↓
    expect_or_drop checks
    ↓
Silver: 9,500 records (500 dropped)
```

**Benefits:**
* ✅ Automatic data cleaning
* ✅ No manual filtering needed
* ✅ Self-documenting quality rules
* ✅ Metrics show what was dropped
* ✅ Downstream tables get clean data

**When to use `expect_or_drop()`:**
* 🥈 **Silver layer** - Primary use case
* 🧹 **Data cleaning** - Remove bad records
* ✅ **Quality enforcement** - Ensure standards
* 📉 **Filtering** - Declarative filtering

**Multiple expectations:**
* Records must pass ALL expectations
* If ANY expectation fails, record is dropped
* AND logic (not OR)

---

**💡 Tip:** `expect_or_drop()` is the most common expectation type for silver layer

### 🧠 Complex Expectation Conditions

Expectations can use complex SQL expressions:

**Multiple conditions (AND):**
```python
expect_or_drop={
    "valid_guest_range": "guests_count > 0 AND guests_count <= 20"
}
```

**Multiple conditions (OR):**
```python
expect_or_drop={
    "valid_status": "status IN ('confirmed', 'completed')"
}
```

**Date validations:**
```python
expect_or_drop={
    "recent_data": "created_at >= '2020-01-01'",
    "valid_date_range": "check_in BETWEEN '2020-01-01' AND '2030-12-31'"
}
```

**NULL checks:**
```python
expect_or_drop={
    "required_field": "column IS NOT NULL",
    "no_empty_strings": "column IS NOT NULL AND column != ''"
}
```

**Calculated conditions:**
```python
expect_or_drop={
    "reasonable_price": "total_amount / nights < 10000",
    "valid_percentage": "discount_pct >= 0 AND discount_pct <= 100"
}
```

**Pattern matching:**
```python
expect_or_drop={
    "valid_email": "email LIKE '%@%.%'",
    "valid_zip": "zip_code RLIKE '^[0-9]{5}$'"
}
```

---

**💡 Tip:** Keep expectations simple and focused - one rule per expectation

### 🎯 Challenge 2: Add expect_or_drop() to Silver Properties

**Your task:**

Create a silver properties table with data quality enforcement.

**Requirements:**

1. Create streaming table: `silver_properties_quality`
2. Read from `bronze_properties`
3. Add these `expect_or_drop` expectations:
   * `valid_price` - base_price > 0 AND base_price < 100000
   * `valid_guests` - max_guests > 0 AND max_guests <= 50
   * `valid_bedrooms` - bedrooms > 0
   * `has_property_type` - property_type IS NOT NULL
4. Add calculated column: `price_per_bedroom` = base_price / bedrooms
5. Select relevant columns

**Expected code:**
```python
@dlt.table(
    name="silver_properties_quality",
    comment="Cleaned properties with quality enforcement",
    expect_or_drop={
        "valid_price": "base_price > 0 AND base_price < 100000",
        "valid_guests": "max_guests > 0 AND max_guests <= 50",
        "valid_bedrooms": "bedrooms > 0",
        "has_property_type": "property_type IS NOT NULL"
    }
)
def silver_properties_quality():
    return (
        dlt.read_stream("bronze_properties")
        .withColumn("price_per_bedroom", col("base_price") / col("bedrooms"))
        .select(
            "property_id", "host_id", "title", "property_type",
            "base_price", "price_per_bedroom", "max_guests", 
            "bedrooms", "bathrooms", "created_at"
        )
    )
```

**After adding:**
1. Add to notebook and save
2. Run pipeline
3. Check graph for the new table
4. View metrics to see how many records were dropped

✅ **Success criteria:** Silver table dropping invalid properties

---

## Part 4: `expect_or_fail()` - Critical Quality Checks 🛑

The `expect_or_fail()` function stops the pipeline if violations occur.

**Use case:** Enforce critical business rules that cannot be violated

---

In [0]:
# ============================================
# GOLD LAYER with expect_or_fail()
# ============================================

@dlt.view(
    name="gold_daily_revenue_critical",
    comment="Daily revenue metrics with critical quality checks - Gold layer",
    expect_or_fail={
        "has_data": "booking_count > 0",
        "positive_revenue": "total_revenue > 0",
        "reasonable_avg": "avg_booking_value > 0 AND avg_booking_value < 50000"
    }
)
def gold_daily_revenue_critical():
    """
    Gold view with expect_or_fail.
    If ANY expectation fails, the ENTIRE PIPELINE stops.
    Use for critical business metrics.
    """
    return (
        dlt.read("silver_bookings_quality")
        .groupBy("booking_date")
        .agg(
            count("booking_id").alias("booking_count"),
            sum("total_amount").alias("total_revenue"),
            avg("total_amount").alias("avg_booking_value"),
            sum("nights").alias("total_nights")
        )
        .orderBy("booking_date")
    )

### 💡 `expect_or_fail()` - How It Works

**What we did:**

1. **Added `expect_or_fail` parameter** to `@dlt.view`
2. **Defined 3 critical expectations:**
   * `has_data` - Must have at least 1 booking per day
   * `positive_revenue` - Revenue must be positive
   * `reasonable_avg` - Average booking value in reasonable range

**What happens:**
* 🛑 If ANY expectation fails, pipeline **stops immediately**
* 🔴 Table marked as **failed**
* ❌ No data written to the table
* 🚨 Requires manual intervention
* 📊 Violation metrics still recorded

**When pipeline fails:**
* Graph shows red node
* Event log shows which expectation failed
* Pipeline status: "Failed"
* Downstream tables don't run
* Must fix issue and rerun

**When to use `expect_or_fail()`:**
* 🥇 **Gold layer** - Critical business metrics
* 🚨 **Regulatory requirements** - Must be met
* 📊 **SLA guarantees** - Data must be valid
* ⚠️ **Critical rules** - Cannot be violated

**Examples of critical checks:**
* Revenue must be positive
* Required fields must exist
* Data must be recent (not too old)
* Aggregations must have minimum records
* Business rules must be satisfied

**Use sparingly:**
* ⚠️ Too many `expect_or_fail` = frequent pipeline failures
* ⚠️ Only for truly critical rules
* ⚠️ Consider `expect_or_drop` first

---

**💡 Tip:** Use `expect_or_fail()` only for critical rules - pipeline stops completely if violated

### 🎯 Challenge 3: Add expect_or_fail() to Gold Metrics

**Your task:**

Create a gold view with critical quality checks.

**Requirements:**

1. Create materialized view: `gold_property_metrics_critical`
2. Read from `silver_bookings_quality`
3. Group by `property_id`
4. Calculate:
   * Total bookings
   * Total revenue
   * Average rating (join with reviews)
5. Add `expect_or_fail` expectations:
   * `min_bookings` - At least 1 booking per property
   * `positive_revenue` - Revenue > 0
   * `valid_avg_rating` - Average rating between 1.0 and 5.0 (or NULL)

**Expected code:**
```python
@dlt.view(
    name="gold_property_metrics_critical",
    comment="Property metrics with critical quality checks",
    expect_or_fail={
        "min_bookings": "booking_count >= 1",
        "positive_revenue": "total_revenue > 0",
        "valid_avg_rating": "avg_rating IS NULL OR (avg_rating >= 1.0 AND avg_rating <= 5.0)"
    }
)
def gold_property_metrics_critical():
    bookings = dlt.read("silver_bookings_quality")
    reviews = dlt.read("silver_reviews")
    
    return (
        bookings
        .groupBy("property_id")
        .agg(
            count("booking_id").alias("booking_count"),
            sum("total_amount").alias("total_revenue"),
            avg("nights").alias("avg_nights")
        )
        .join(
            reviews.groupBy("property_id").agg(avg("rating").alias("avg_rating")),
            "property_id",
            "left"
        )
    )
```

**After adding:**
1. Add to notebook and save
2. Run pipeline
3. Verify table appears in graph
4. Check if pipeline succeeds (all expectations pass)
5. Try modifying an expectation to make it fail (for testing)

✅ **Success criteria:** Gold view with critical expectations

---

## Part 5: Combining Multiple Expectation Types 🔗

You can use multiple expectation types on the same table!

**Why combine?**
* Different severity levels for different rules
* Track some issues, drop others, fail on critical
* Flexible data quality strategy

---

In [0]:
# ============================================
# COMBINING MULTIPLE EXPECTATION TYPES
# ============================================

@dlt.table(
    name="silver_bookings_comprehensive",
    comment="Bookings with comprehensive data quality checks",
    # Track these issues (keep records)
    expect={
        "recent_booking": "created_at >= '2023-01-01'",
        "reasonable_nights": "nights <= 365"
    },
    # Drop records with these issues
    expect_or_drop={
        "valid_amount": "total_amount > 0",
        "valid_guests": "guests_count > 0 AND guests_count <= 20",
        "valid_dates": "check_in < check_out",
        "confirmed_only": "status = 'confirmed'"
    }
    # Note: expect_or_fail not used here (too strict for silver)
)
def silver_bookings_comprehensive():
    """
    Silver table with multiple expectation types:
    - expect: Track old bookings and long stays (warnings)
    - expect_or_drop: Remove invalid data (filtering)
    """
    return (
        dlt.read_stream("bronze_bookings")
        .withColumn("nights", datediff(col("check_out"), col("check_in")))
        .withColumn("booking_date", to_date(col("created_at")))
        .select(
            "booking_id",
            "user_id",
            "property_id",
            "check_in",
            "check_out",
            "nights",
            "guests_count",
            "total_amount",
            "status",
            "booking_date",
            "created_at"
        )
    )

### 💡 Combining Expectations - How It Works

**What we did:**

1. **Used both `expect` and `expect_or_drop`** on the same table
2. **Different actions for different rules:**
   * `expect` - Track old bookings (warning, keep data)
   * `expect_or_drop` - Remove invalid data (filtering)

**Execution order:**
```
1. Read data from bronze
   ↓
2. Apply transformations (withColumn, select)
   ↓
3. Check expect_or_drop rules
   ↓
4. Drop records that fail expect_or_drop
   ↓
5. Check expect rules on remaining records
   ↓
6. Track violations, keep all remaining records
   ↓
7. Write to silver table
```

**Result:**
* Invalid records dropped (expect_or_drop)
* Warnings tracked for remaining records (expect)
* Clean data in output
* Full visibility into data quality

**Strategy by severity:**

| Severity | Expectation Type | Example |
|----------|-----------------|----------|
| **Info** | `expect` | Old data, unusual patterns |
| **Warning** | `expect_or_drop` | Invalid values, nulls |
| **Critical** | `expect_or_fail` | Business rule violations |

**Best practice pattern:**
```python
@dlt.table(
    expect={"info_level": "condition"},           # Track
    expect_or_drop={"warning_level": "condition"}, # Filter
    expect_or_fail={"critical_level": "condition"} # Halt
)
```

---

**💡 Tip:** Combine expectation types to create a comprehensive data quality strategy

### 🎯 Challenge 4: Comprehensive Quality Checks

**Your task:**

Create a silver table with all three expectation types.

**Requirements:**

1. Create streaming table: `silver_reviews_comprehensive`
2. Read from `bronze_reviews`
3. Add **all three** expectation types:

**expect (track):**
* `has_comment` - comment IS NOT NULL (track reviews without comments)
* `long_comment` - LENGTH(comment) > 10 (track short comments)

**expect_or_drop (filter):**
* `valid_rating` - rating BETWEEN 1.0 AND 5.0
* `not_deleted` - is_deleted = false
* `has_property` - property_id IS NOT NULL

**expect_or_fail (critical):**
* `has_data` - review_id IS NOT NULL (critical - must have ID)

4. Add calculated columns:
   * `review_date` = DATE(created_at)
   * `comment_length` = LENGTH(comment)

**Expected code:**
```python
@dlt.table(
    name="silver_reviews_comprehensive",
    comment="Reviews with comprehensive quality checks",
    expect={
        "has_comment": "comment IS NOT NULL",
        "long_comment": "LENGTH(comment) > 10"
    },
    expect_or_drop={
        "valid_rating": "rating BETWEEN 1.0 AND 5.0",
        "not_deleted": "is_deleted = false",
        "has_property": "property_id IS NOT NULL"
    },
    expect_or_fail={
        "has_data": "review_id IS NOT NULL"
    }
)
def silver_reviews_comprehensive():
    return (
        dlt.read_stream("bronze_reviews")
        .withColumn("review_date", to_date(col("created_at")))
        .withColumn("comment_length", length(col("comment")))
        .select(
            "review_id", "booking_id", "property_id", "user_id",
            "rating", "comment", "comment_length", "review_date", "created_at"
        )
    )
```

**After adding:**
1. Add to notebook and save
2. Run pipeline
3. View data quality metrics
4. Check how many records were dropped
5. Check warning counts

✅ **Success criteria:** Table with all 3 expectation types working

---

## Part 6: Monitoring Data Quality Metrics 📊

Lakeflow automatically tracks and displays data quality metrics.

---

### 📊 Step 1: View Expectations in Pipeline Graph

**Follow these steps:**

1. Run your pipeline with expectations
2. Go to the **"Graph"** tab
3. Look at nodes with expectations

**What you'll see:**

**Nodes with expectations show:**
* ✅ **Green checkmark** - All expectations passed
* ⚠️ **Yellow warning** - Some expect() violations (tracked)
* 🔴 **Red X** - expect_or_fail violations (failed)
* 📊 **Metrics badge** - Number of expectations

**Click on a node:**
* Side panel opens
* Shows **"Data Quality"** section
* Lists all expectations
* Shows pass/fail status
* Shows violation counts

**Try this:**
1. Run pipeline
2. Click on `silver_bookings_quality` node
3. Look for "Data Quality" or "Expectations" section
4. See metrics for each expectation

✅ **Checkpoint:** Can view expectation metrics in graph

---

### 📈 Step 2: Understand Expectation Metrics

**For each expectation, you'll see:**

**Metrics displayed:**
* 📊 **Total records** - Records processed
* ❌ **Violations** - Records that failed the check
* 📊 **Violation %** - Percentage of failures
* ✅ **Passed** - Records that passed
* 🎯 **Action taken** - Tracked, dropped, or failed

**Example metrics:**
```
Expectation: valid_amount
Condition: total_amount > 0
Total records: 10,000
Violations: 150 (1.5%)
Action: Dropped
Passed: 9,850
```

**Interpreting metrics:**

**Low violation rate (< 1%):**
* ✅ Good data quality
* ✅ Expectations are appropriate
* ✅ No action needed

**Medium violation rate (1-5%):**
* ⚠️ Some data quality issues
* 🔍 Investigate root cause
* 📊 Monitor trend over time

**High violation rate (> 5%):**
* 🔴 Significant data quality issues
* 🚨 Investigate immediately
* 🔧 May need to fix upstream processes
* ⚠️ Or expectation is too strict

---

**💡 Tip:** Monitor violation trends over time - increasing violations indicate upstream issues

### 📜 Step 3: View Expectations in Event Log

**Follow these steps:**

1. Go to the **"Event log"** tab
2. Find the table execution entry
3. Expand to see details

**Event log shows:**
* 📅 **Timestamp** - When table ran
* ⏱️ **Duration** - Execution time
* 📊 **Rows processed** - Input records
* 📊 **Rows output** - Output records (after drops)
* ✅ **Expectations** - All expectation results

**For each expectation:**
```
Expectation: valid_amount
  Status: Passed (with violations)
  Records processed: 10,000
  Violations: 150
  Violation rate: 1.5%
  Action: Dropped 150 records
```

**Failed expectations:**
```
Expectation: has_data
  Status: FAILED
  Records processed: 0
  Violations: 100%
  Action: Pipeline stopped
  Error: No records passed expectation
```

**Try this:**
1. Open Event log
2. Find your table execution
3. Review expectation metrics
4. Compare input vs output row counts

✅ **Checkpoint:** Can view detailed expectation metrics in event log

---

**💡 Tip:** Event log is essential for debugging expectation failures

### 🔍 Step 4: Query Expectation Metrics

**Lakeflow stores expectation metrics** that you can query!

**System tables:**

Lakeflow creates system tables with metrics:
* `system.lakeflow.event_log` - Pipeline execution events
* Table-specific metrics in pipeline metadata

**Query expectation metrics:**
```sql
-- View data quality metrics from event log
SELECT 
  timestamp,
  details:flow_definition.output_dataset AS table_name,
  details:flow_definition.expectations AS expectations,
  details:flow_progress.metrics.num_output_rows AS output_rows
FROM event_log(TABLE(your_pipeline_name))
WHERE details:flow_definition.expectations IS NOT NULL
ORDER BY timestamp DESC
```

**Analyze quality trends:**
```sql
-- Track violation rates over time
SELECT 
  DATE(timestamp) AS date,
  table_name,
  expectation_name,
  SUM(violations) AS total_violations,
  AVG(violation_rate) AS avg_violation_rate
FROM expectation_metrics
GROUP BY DATE(timestamp), table_name, expectation_name
ORDER BY date DESC
```

**Benefits:**
* 📊 Track data quality over time
* 📈 Create quality dashboards
* 🚨 Alert on quality degradation
* 🔍 Root cause analysis

---

**💡 Tip:** Build dashboards from expectation metrics to monitor data quality trends

## 📚 Expectations Best Practices

### **Choosing Expectation Types**

**By layer:**

| Layer | Primary Type | Reason |
|-------|-------------|--------|
| Bronze | `expect()` | Track issues, keep all data |
| Silver | `expect_or_drop()` | Filter bad data |
| Gold | `expect_or_fail()` | Critical metrics only |

**By severity:**

| Severity | Type | Example |
|----------|------|----------|
| **Info** | `expect()` | Unusual patterns, old data |
| **Warning** | `expect_or_drop()` | Invalid values, nulls |
| **Critical** | `expect_or_fail()` | Business rules, SLAs |

---

### **Writing Good Expectations**

**Do:**
* ✅ Use descriptive names: `valid_amount`, not `check1`
* ✅ One rule per expectation
* ✅ Simple, clear conditions
* ✅ Test expectations before deploying
* ✅ Document why expectation exists

**Don't:**
* ❌ Complex multi-condition checks in one expectation
* ❌ Too many expectations (5-10 max per table)
* ❌ Expectations that always fail
* ❌ Duplicate checks across layers

---

### **Expectation Naming**

**Good names:**
* `valid_amount` - Clear what it checks
* `positive_price` - Descriptive
* `required_email` - Indicates requirement
* `valid_date_range` - Explains validation

**Bad names:**
* `check1` - Not descriptive
* `test` - Unclear purpose
* `exp` - Too generic

---

### **Performance Considerations**

**Expectations impact performance:**
* Each expectation = additional computation
* Complex conditions = slower processing
* Many expectations = more overhead

**Optimize:**
* ✅ Keep conditions simple
* ✅ Limit to 5-10 expectations per table
* ✅ Use indexed columns when possible
* ✅ Combine related checks

---

### **Testing Expectations**

**Before production:**
1. Start with `expect()` to understand violations
2. Review violation rates
3. Adjust thresholds if needed
4. Change to `expect_or_drop()` when ready
5. Use `expect_or_fail()` only for critical rules

**Iterative approach:**
```
1. expect() - Learn about data
   ↓
2. Analyze violation rates
   ↓
3. expect_or_drop() - Clean data
   ↓
4. Monitor for issues
   ↓
5. expect_or_fail() - Critical rules only
```

---

## 📊 Complete Pipeline with Expectations

Here's a complete example showing expectations at each layer:

### **Bronze Layer - Track Issues**
```python
@dlt.table(
    expect={
        "recent_data": "created_at >= '2020-01-01'",
        "has_amount": "total_amount IS NOT NULL"
    }
)
def bronze_bookings():
    return spark.readStream.table("samples.wanderbricks.bookings")
```
**Action:** Track violations, keep all records

---

### **Silver Layer - Clean Data**
```python
@dlt.table(
    expect={
        "reasonable_nights": "nights <= 365"  # Track long stays
    },
    expect_or_drop={
        "valid_amount": "total_amount > 0",
        "valid_guests": "guests_count > 0",
        "valid_dates": "check_in < check_out",
        "confirmed": "status = 'confirmed'"
    }
)
def silver_bookings():
    return (
        dlt.read_stream("bronze_bookings")
        .withColumn("nights", datediff(col("check_out"), col("check_in")))
        .filter(col("total_amount").isNotNull())
    )
```
**Action:** Track long stays, drop invalid records

---

### **Gold Layer - Critical Checks**
```python
@dlt.view(
    expect_or_fail={
        "has_data": "booking_count > 0",
        "positive_revenue": "total_revenue > 0"
    }
)
def gold_daily_revenue():
    return (
        dlt.read("silver_bookings")
        .groupBy("booking_date")
        .agg(
            count("*").alias("booking_count"),
            sum("total_amount").alias("total_revenue")
        )
    )
```
**Action:** Fail pipeline if no data or negative revenue

---

### **Result:**
* 🥉 Bronze: All data kept, issues tracked
* 🥈 Silver: Clean data, invalid records dropped
* 🥇 Gold: Critical quality guaranteed

---

## 📊 Common Expectation Patterns

### **Pattern 1: Required Fields**
```python
expect_or_drop={
    "has_id": "id IS NOT NULL",
    "has_timestamp": "timestamp IS NOT NULL",
    "has_amount": "amount IS NOT NULL"
}
```

---

### **Pattern 2: Value Ranges**
```python
expect_or_drop={
    "valid_rating": "rating BETWEEN 1 AND 5",
    "valid_percentage": "percentage >= 0 AND percentage <= 100",
    "positive_amount": "amount > 0"
}
```

---

### **Pattern 3: Date Validations**
```python
expect_or_drop={
    "valid_date_order": "start_date < end_date",
    "recent_data": "created_at >= '2020-01-01'",
    "not_future": "created_at <= CURRENT_TIMESTAMP()"
}
```

---

### **Pattern 4: Business Rules**
```python
expect_or_drop={
    "confirmed_bookings": "status = 'confirmed'",
    "valid_payment": "payment_status = 'completed'",
    "active_users": "is_active = true"
}
```

---

### **Pattern 5: Referential Integrity**
```python
expect_or_drop={
    "has_user": "user_id IS NOT NULL",
    "has_product": "product_id IS NOT NULL",
    "valid_foreign_key": "customer_id IN (SELECT id FROM customers)"
}
```

---

### **Pattern 6: Data Completeness**
```python
expect={
    "has_description": "description IS NOT NULL",
    "has_category": "category IS NOT NULL",
    "has_tags": "tags IS NOT NULL"
}
```

---

### **Pattern 7: Aggregation Validation**
```python
expect_or_fail={
    "min_records": "record_count >= 100",
    "positive_total": "total_amount > 0",
    "valid_average": "avg_value > 0"
}
```

---

**💡 Tip:** Build a library of common expectation patterns for your organization

## 🔧 Troubleshooting Expectations

**Expectation always failing:**
* ✅ Check SQL condition syntax
* ✅ Verify column names are correct
* ✅ Test condition in regular SQL query first
* ✅ Check if condition is too strict
* ✅ Review sample data

**High violation rates:**
* ✅ Review expectation condition
* ✅ Check if upstream data changed
* ✅ Analyze violated records
* ✅ Adjust threshold if needed
* ✅ Fix upstream data quality

**Pipeline failing with expect_or_fail:**
* ✅ Check event log for details
* ✅ Review which expectation failed
* ✅ Query source data to understand issue
* ✅ Consider changing to expect_or_drop temporarily
* ✅ Fix root cause

**Expectations not showing in UI:**
* ✅ Verify expectation syntax is correct
* ✅ Check decorator parameters
* ✅ Ensure pipeline ran successfully
* ✅ Refresh the page

**Performance issues:**
* ✅ Reduce number of expectations
* ✅ Simplify complex conditions
* ✅ Use indexed columns
* ✅ Profile slow expectations

---

**💡 Tip:** Test expectations on sample data before applying to full pipeline

## 📊 Quick Reference Guide

### **Expectation Types**

```python
# Track violations (keep records)
@dlt.table(
    expect={"name": "condition"}
)

# Drop invalid records
@dlt.table(
    expect_or_drop={"name": "condition"}
)

# Fail pipeline on violations
@dlt.table(
    expect_or_fail={"name": "condition"}
)

# Combine multiple types
@dlt.table(
    expect={"track": "condition"},
    expect_or_drop={"drop": "condition"},
    expect_or_fail={"critical": "condition"}
)
```

---

### **Common Conditions**

```python
# NOT NULL
"column IS NOT NULL"

# Positive values
"amount > 0"

# Range
"value BETWEEN 1 AND 100"

# Multiple conditions
"amount > 0 AND amount < 10000"

# IN list
"status IN ('active', 'pending')"

# Date comparison
"start_date < end_date"

# Pattern matching
"email LIKE '%@%.%'"
```

---

### **By Layer**

**Bronze:**
```python
expect={"track_issues": "condition"}
```

**Silver:**
```python
expect_or_drop={"filter_invalid": "condition"}
```

**Gold:**
```python
expect_or_fail={"critical_rule": "condition"}
```

---

### **Monitoring**

* **Graph tab:** Visual expectation status
* **Node details:** Violation counts and percentages
* **Event log:** Detailed execution metrics
* **System tables:** Query historical metrics

---

## 🎉 Congratulations!

You've completed the Lakeflow Expectations demo!

### **What You Learned:**

✅ **Expectations concept** - Declarative data quality rules  
✅ **expect()** - Track violations, keep records  
✅ **expect_or_drop()** - Filter invalid data  
✅ **expect_or_fail()** - Halt on critical issues  
✅ **Combining expectations** - Multiple types on one table  
✅ **Monitoring metrics** - View quality in UI  
✅ **Best practices** - When and how to use each type  
✅ **Common patterns** - Reusable expectation templates  

---

### **Key Takeaways:**

1. **Expectations are declarative** - Define rules, Lakeflow enforces
2. **Three types for three actions** - Track, drop, or fail
3. **Bronze tracks, silver drops, gold fails** - General pattern
4. **Metrics are automatic** - No manual tracking needed
5. **Combine types for flexibility** - Different rules, different actions
6. **Start lenient, get stricter** - expect → expect_or_drop → expect_or_fail
7. **Monitor trends** - Increasing violations = upstream issues

---

### **Your Data Quality Strategy:**

```
🥉 Bronze Layer
   expect() - Track all issues
   Keep all data for audit
   ↓
🥈 Silver Layer
   expect() - Track unusual patterns
   expect_or_drop() - Remove invalid data
   Clean data flows downstream
   ↓
🥇 Gold Layer
   expect_or_fail() - Critical business rules
   Guarantee data quality
   ↓
📊 Dashboards & Analytics
   High-quality, trusted data
```

---

### **Expectation Decision Tree:**

```
Is this rule critical?
  │
  ├─ YES → expect_or_fail() (Gold layer)
  │
  └─ NO → Should invalid records be removed?
           │
           ├─ YES → expect_or_drop() (Silver layer)
           │
           └─ NO → expect() (Bronze layer, monitoring)
```

---

### **Production Checklist:**

☐ Expectations defined for all layers  
☐ Expectation names are descriptive  
☐ Conditions are tested and validated  
☐ Violation rates are acceptable  
☐ Critical rules use expect_or_fail  
☐ Monitoring dashboard created  
☐ Alert thresholds set  
☐ Team trained on expectation failures  
☐ Runbook for handling failures  
☐ Regular review of quality metrics  

---

### **Next Steps:**

* Add expectations to your existing pipelines
* Build data quality dashboards
* Set up alerts for quality degradation
* Create expectation libraries
* Document data quality standards
* Train team on expectation patterns
* Explore advanced DLT features

---

### **Resources:**

* [Delta Live Tables Expectations](https://docs.databricks.com/delta-live-tables/expectations.html)
* [Data Quality Best Practices](https://docs.databricks.com/delta-live-tables/expectations.html#best-practices)
* [Monitoring DLT Pipelines](https://docs.databricks.com/delta-live-tables/observability.html)
* [DLT Python API Reference](https://docs.databricks.com/delta-live-tables/python-ref.html)

---

**You're now ready to build production pipelines with robust data quality!** 🚀

*Happy pipeline building with confidence!*